In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [11]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312      1.0 -6.702704e-04                  NaN       NaN
0.000312      NaN           NaN                  1.0 -0.000033
0.000750      2.0 -6.699875e-04                  2.0 -0.000037
0.001052      3.0 -6.689541e-04                  3.0 -0.000029
0.001476      4.0 -6.676074e-04                  4.0 -0.000027
0.002070      5.0 -6.660580e-04                  5.0 -0.000025
0.002904      6.0 -6.644343e-04                  6.0 -0.000024
0.004074      7.0 -6.628377e-04                  7.0 -0.000022
0.005714      8.0 -6.610948e-04                  8.0 -0.000020
0.008015      9.0 -6.595067e-04                  9.0 -0.000018
0.011243     10.0 -6.579689e-04                 10.0 -0.000016
0.015771     11.0 -6.570496e-04                 11.0 -0.000013
0.022122     12.0 -6.568870e-04                 12.0 -0.000011
0.031031     13.0 -6.578162e-04                 13.0 -0.000008
0.043528     14.0 -6.601551e-04                 14.0 -0.000006
0.061057     15.0 -6.548656e-04                 15.0  0.000001
0.085645     16.0 -6.109366e-04                 16.0  0.000032
0.120136     17.0 -4.915039e-04                 17.0  0.000097
0.168516     18.0 -2.811832e-04                 18.0  0.000202
0.236378     19.0  5.629998e-05                 19.0  0.000366
0.331549     20.0  5.728185e-04                 20.0  0.000618
0.465100     21.0  1.339775e-03                 21.0  0.000996
0.652400     22.0  2.453328e-03                 22.0  0.001552
0.915100     23.0  3.635652e-03                 23.0  0.002166
1.283650     24.0  3.613053e-03                 24.0  0.002227
1.800600     25.0  2.495162e-03                 25.0  0.001735
2.525700     26.0  1.495763e-03                 26.0  0.001237
3.542800     27.0  8.300265e-04                 27.0  0.000854
4.969550     28.0  4.328873e-04                 28.0  0.000577
6.970850     29.0  2.221521e-04                 29.0  0.000387
9.778100     30.0  1.151002e-04                 30.0  0.000255
13.715850    31.0  5.448747e-05                 31.0  0.000158
19.239350    32.0  1.799927e-05                 32.0  0.000087
26.987250    33.0  3.654192e-06                 33.0  0.000041
37.855300    34.0 -4.652111e-07                 34.0  0.000012
53.100050    35.0 -9.582664e-06                 35.0 -0.000013
73.887500    36.0 -2.361337e-05                 36.0 -0.000035
97.662500    37.0 -3.341440e-05                 37.0 -0.000050
121.437500   38.0 -3.944783e-05                 38.0 -0.000059
145.212500   39.0 -4.915230e-05                 39.0 -0.000068
168.987500   40.0 -5.901181e-05                 40.0 -0.000077
192.762500   41.0 -7.455693e-05                 41.0 -0.000103
216.537500   42.0 -1.064638e-04                 42.0 -0.000189
240.312500   43.0 -1.220898e-04                 43.0 -0.000305
264.087500   44.0 -1.569970e-04                 44.0 -0.000505
287.862500   45.0 -2.566387e-04                 45.0 -0.000676
311.637500   46.0 -3.230123e-04                 46.0 -0.000659
335.412500   47.0 -2.668091e-04                 47.0 -0.000625
359.187500   48.0 -8.240387e-05                 48.0 -0.000655
382.962500   49.0  2.318333e-04                 49.0 -0.000723
406.737500   50.0  6.805777e-04                 50.0 -0.000716
430.512500   51.0  1.253684e-03                 51.0 -0.000468
454.287500   52.0  1.925891e-03                 52.0  0.000153
478.062500   53.0  2.726550e-03                 53.0  0.001211
501.837500   54.0  3.676744e-03                 54.0  0.002622
525.612500   55.0  4.786407e-03                 55.0  0.004370
549.387500   56.0  6.080320e-03                 56.0  0.006374
573.162500   57.0  7.611592e-03                 57.0  0.008546
596.937500   58.0  9.679464e-03                 58.0  0.011139
620.712500   59.0  1.181076e-02                 59.0  0.013461
644.487500   60.0  1.393085e-02              

# Fluxes by Layer

CRD                            CLIRAD            \
                        flug          fldg     fnetg      flug      fldg   
pressure     level                                                         
0.000000e+00 1     -5.098632  0.000000e+00 -5.098632       NaN       NaN   
1.000000e-08 1           NaN           NaN       NaN -4.873729  0.000002   
6.244000e-04 2     -5.098632  5.618312e-09 -5.098632 -4.873729  0.000002   
8.759000e-04 3     -5.098632  7.921962e-09 -5.098632 -4.873729  0.000002   
1.228600e-03 4     -5.098632  1.127727e-08 -5.098632 -4.873729  0.000002   
1.723400e-03 5     -5.098632  1.617629e-08 -5.098632 -4.873729  0.000002   
2.417400e-03 6     -5.098632  2.332701e-08 -5.098632 -4.873729  0.000002   
3.390900e-03 7     -5.098632  3.376440e-08 -5.098632 -4.873729  0.000002   
4.756500e-03 8     -5.098632  4.898635e-08 -5.098632 -4.873729  0.000002   
6.672000e-03 9     -5.098632  7.117703e-08 -5.098632 -4.873729  0.000002   
9.358900e-03 10    -5.098632  1.035146e-07 -5.098632 -4.873729  0.000002   
1.312780e-02 11    -5.098633  1.505885e-07 -5.098633 -4.873729  0.000002   
1.841450e-02 12    -5.098633  2.190148e-07 -5.098633 -4.873729  0.000002   
2.583020e-02 13    -5.098634  3.183752e-07 -5.098634 -4.873729  0.000002   
3.623230e-02 14    -5.098635  4.623887e-07 -5.098634 -4.873729  0.000002   
5.082340e-02 15    -5.098636  6.712033e-07 -5.098636 -4.873729  0.000002   
7.129060e-02 16    -5.098638  9.989392e-07 -5.098637 -4.873729  0.000002   
1.000000e-01 17    -5.098641  1.626744e-06 -5.098639 -4.873729  0.000003   
1.402710e-01 18    -5.098644  2.978889e-06 -5.098642 -4.873729  0.000003   
1.967600e-01 19    -5.098649  5.859960e-06 -5.098643 -4.873729  0.000004   
2.759970e-01 20    -5.098655  1.188057e-05 -5.098643 -4.873728  0.000006   
3.871000e-01 21    -5.098660  2.421211e-05 -5.098635 -4.873725  0.000011   
5.431000e-01 22    -5.098660  4.898738e-05 -5.098611 -4.873717  0.000022   
7.617000e-01 23    -5.098645  9.748603e-05 -5.098547 -4.873698  0.000044   
1.068500e+00 24    -5.098598  1.826684e-04 -5.098415 -4.873661  0.000085   
1.498800e+00 25    -5.098521  2.904318e-04 -5.098231 -4.873608  0.000146   
2.102400e+00 26    -5.098439  3.864776e-04 -5.098052 -4.873550  0.000212   
2.949000e+00 27    -5.098360  4.578924e-04 -5.097902 -4.873494  0.000279   
4.136600e+00 28    -5.098289  5.030207e-04 -5.097786 -4.873440  0.000346   
5.802500e+00 29    -5.098225  5.250968e-04 -5.097700 -4.873392  0.000412   
8.139200e+00 30    -5.098170  5.308380e-04 -5.097639 -4.873349  0.000476   
1.141700e+01 31    -5.098121  5.267877e-04 -5.097594 -4.873314  0.000540   
1.601470e+01 32    -5.098082  5.172109e-04 -5.097564 -4.873289  0.000601   
2.246400e+01 33    -5.098057  5.064647e-04 -5.097551 -4.873280  0.000658   
3.151050e+01 34    -5.098050  5.033718e-04 -5.097547 -4.873290  0.000713   
4.420010e+01 35    -5.098064  5.163816e-04 -5.097547 -4.873328  0.000768   
6.200000e+01 36    -5.098114  5.468004e-04 -5.097568 -4.873409  0.000822   
8.577500e+01 37    -5.098229  5.945743e-04 -5.097634 -4.873558  0.000870   
1.095500e+02 38    -5.098382  6.533053e-04 -5.097728 -4.873738  0.000909   
1.333250e+02 39    -5.098564  7.251528e-04 -5.097839 -4.873945  0.000949   
1.571000e+02 40    -5.098782  8.039059e-04 -5.097978 -4.874178  0.000990   
1.808750e+02 41    -5.099044  8.999134e-04 -5.098144 -4.874444  0.001041   
2.046500e+02 42    -5.099465  1.111467e-03 -5.098354 -4.874861  0.001166   
2.284250e+02 43    -5.100509  1.854841e-03 -5.098654 -4.875924  0.001696   
2.522000e+02 44    -5.102678  3.680897e-03 -5.098998 -4.878189  0.003104   
2.759750e+02 45    -5.106697  7.256927e-03 -5.099440 -4.882455  0.005946   
2.997500e+02 46    -5.113538  1.337575e-02 -5.100162 -4.889791  0.011377   
3.235250e+02 47    -5.122907  2.183442e-02 -5.101072 -4.899878  0.019609   
3.473000e+02 48    -5.134752  3.292813e-02 -5.101824 -4.912714  0.030684   
3.710750e+02 49    -5.149378  4.732250e-02 -5.102056 -4.928666  0.044791 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')